In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段2：实验：开始使用预构建的TFHub模型

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_tfhub_models.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub标志">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_tfhub_models.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab标志"> 在Colab中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_with_tfhub_models.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI标志">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的MLOps。本教程涵盖了第二阶段：实验：开始使用预构建的TensorFlow Hub（TFHub）模型。

### 目标

在本教程中，您将学习如何使用来自TensorFlow Hub的预构建模型来使用 `Vertex AI Training`。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Training`
- `Vertex AI Model` 资源

执行的步骤包括：

- 下载一个TensorFlow Hub预构建模型。
- 将任务组件添加为CIFAR-10数据集的分类器。
- 使用迁移学习在本地微调模型训练。
- 构建自定义训练脚本：
    - 从TensorFlow Datasets获取训练数据
    - 从TensorFlow Hub获取模型架构
    - 训练模型
    - 保存模型文件并上传为Vertex AI Model资源。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。数据集的版本已经内置到了TensorFlow中。训练好的模型预测图片属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install tensorflow-datasets $USER_FLAG -q
! pip3 install -U google-cloud-storage $USER_FLAG -q

### 重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*如果有这个选项，请确保在 GPU 运行时下运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额用于支付计算/存储成本。

2. [确保为您的项目启用了计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI APIs、Compute Engine APIs 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您正在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行此单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!` 为前缀的行视为 shell 命令，并会插入以 `$` 为前缀的 Python 变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于本笔记本的操作。以下是支持Vertex AI的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不会在使用Vertex AI进行训练时使用多区域存储桶。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在直播教程会话中，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证你的Google Cloud账号

**如果你正在使用Vertex AI工作台笔记本**，你的环境已经通过验证。跳过这一步。

**如果你正在使用Colab**，运行下面的单元格，并根据提示进行身份验证通过oAuth。

**否则**，按照以下步骤操作：

在Cloud控制台中，进入[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务账号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"存储对象管理员"，并选择**存储对象管理员**。

点击创建。一个包含你密钥的JSON文件将下载到你的本地环境。

在下面的单元格中输入你服务账号密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用何种记事本环境，以下步骤均为必需。**

当您使用Python的Vertex客户端库提交自定义训练作业时，您需要将包含训练代码的Python包上传到云存储桶中。Vertex AI将从此包中运行代码。在本教程中，Vertex AI还会将您作业的训练模型保存在同一个存储桶中。然后您可以根据此输出在`Endpoint`资源中创建一个端点来提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对Cloud Storage存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入 Vertex AI SDK
将 Vertex AI SDK 导入到您的 Python 环境中。

In [ ]:
import google.cloud.aiplatform as aip

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像以及分配给虚拟机实例（VM）的GPU数量。例如，要使用一个具有4个Nvidia Telsa K80 GPU的GPU容器映像分配到每个VM中，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定`(None, None)`以使用在CPU上运行的容器映像。

了解更多关于您所在地区的 [硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：在不支持GPU的TF 2.3之前的TF版本中，将无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由生成服务函数中的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于训练和预测的预先构建的Docker容器镜像。 

有关最新列表，请参阅[用于训练的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。 

有关最新列表，请参阅[用于预测的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.1".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 设置变量`TRAIN_COMPUTE`来配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]个vCPU的数量

*注意：以下内容不支持用于训练：*

 - `standard`：2个vCPU
 - `highcpu`：2、4和8个vCPU

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

### 检查培训包

#### 包布局 

在开始培训之前，您将看一下如何为自定义培训任务组装 Python 包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py
  - data.py
  - model.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 映像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义培训任务的 Python 脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为点号 (`trainer.task`) 并且去掉文件后缀 (`.py`)。

#### 包组装

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = """import setuptools\n\nsetuptools.setup( \
    install_requires=[       
        'tensorflow==2.5.0',      
        'tensorflow_datasets==1.3.0',
        'tensorflow-hub'
    ],
    packages=setuptools.find_packages())"""
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR-10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

TensorFlow Hub 模型介绍

TensorFlow Hub 是一个预训练模型的存储库。对于每个预训练的大型模型，可以选择以下之一：

- 完整训练的大型模型（即，云模型）-- 带有任务组件。
- 完整训练的大型模型 -- 没有任务组件。

在后一种情况下，模型的输出是低维度的嵌入向量，在 TensorFlow Hub 中被称为特征向量。对于图像模型，输入形状也可以被覆盖。

这些特征向量模型可以以单个 TF.Keras 层的格式下载，使用方法 `hub.KerasLayer()`，可以从中构建一个新模型。

了解更多关于[TensorFlow Hub](https://www.tensorflow.org/hub)。

获取训练数据

接下来，您可以编写 Python 模块 `data.py` 来获取训练数据。

In [ ]:
%%writefile custom/trainer/data.py

import tensorflow as tf
import tensorflow_datasets as tfds

def get_data(buffer=10000):
  
    # Scaling CIFAR10 data from (0, 255] to (0., 1.]
    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255.0
        return image, label


    datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True
    )
    
    return datasets['train'].map(scale).cache().shuffle(buffer).repeat()

#### 在本地测试get_data()模块

接下来，您可以通过导入模块并调用`get_data()`函数来测试您刚刚编写的模块。 这将返回一个用于CIFAR-10训练数据的TFDS生成器。

In [ ]:
os.chdir("custom/trainer")
from data import get_data

train_dataset = get_data().batch(64)

os.chdir("../..")

### 从TFHub获取预训练模型

接下来，您需要编写名为`model.py`的模块来获取模型的架构。该模块执行以下操作：

- 接受以下参数：
    - `tfhub_url`：TensorFlow Hub模型的URL。
    - `input_shape`：模型的输入形状。
    - `n_classes`：分类的输出类别数量。
    - `lr`：编译模型的学习率。
- 下载TensorFlow Hub模型。
- 冻结TensorFlow Hub模型层的权重 -- 不会重新训练。
- 添加分类任务组件 -- 这些层将被训练。
- 编译模型。

In [ ]:
%%writefile custom/trainer/model.py

import tensorflow as tf
import tensorflow_hub as hub

def get_model(tfhub_url, input_shape, n_classes, lr=0.001):

    tfhub_model = tf.keras.Sequential([
        hub.KerasLayer(tfhub_url,
                       input_shape=input_shape)
    ])
    
    tfhub_model.trainable = False
    
    model = tf.keras.Sequential([
        tfhub_model,
        tf.keras.layers.Dense(n_classes, activation='softmax')
    ])
    
    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.SGD(learning_rate=lr),
        metrics=['accuracy']
    )
    
    return model

测试本地get_model()模块

下一步，你要测试刚刚写的模块，通过导入模块并调用`get_model()`函数。这将返回一个为CIFAR-10数据集配置的TF.Keras模型。

In [ ]:
os.chdir("custom/trainer")

from model import get_model

model = get_model(
    tfhub_url="https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/4",
    input_shape=(32, 32, 3),
    n_classes=10,
    lr=0.001,
)

os.chdir("../..")

### 训练模型

接下来，您需要编写 `train.py` 模块来训练模型。

In [ ]:
%%writefile custom/trainer/train.py

def train_model(model,
                train_dataset,
                epochs,
                steps_per_epoch
               ):

    history = model.fit(train_dataset, epochs=epochs, steps_per_epoch=steps_per_epoch)
    return model

### 在本地测试train_model()模块

接下来，通过导入模块并调用`train_model()`函数来测试您刚刚编写的模块。 此函数将返回经过训练的模型。

In [ ]:
os.chdir("custom/trainer")

from train import train_model

model = train_model(model, train_dataset, 10, 200)

os.chdir("../..")

### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。

- 解析用于训练模型的命令行参数
    - `--model-dir`: 从命令行 (`--model_dir`) 获取保存模型文件的目录，如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
    - `--lr`: 学习率。
    - `--epochs`: 训练的轮数。
    - `--steps`: 每轮的步数。
    - `--batch_size`: 每个 worker 的批量大小。


- `get_data()`: 下载并预处理 CIFAR-10 数据集。
- `get_model()`: 从 TFHub 下载模型架构并配置为 CIFAR-10 训练。
- `train_model()`: 训练模型。
- 将模型文件保存到指定的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
import argparse
import os
import sys
import logging
tfds.disable_progress_bar()

from . import data
from . import model as model_
from . import train

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--batch_size', dest='batch_size',
                    default=64, type=int,
                    help='Size of mini-batch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
    logging.info("Single device training")
# Single Machine, multiple compute device
elif args.distribute == 'mirrored':
    strategy = tf.distribute.MirroredStrategy()
    logging.info("Mirrored Strategy distributed training")
# Multi Machine, multiple compute device
elif args.distribute == 'multiworker':
    strategy = tf.distribute.MultiWorkerMirroredStrategy()
    logging.info("Multi-worker Strategy distributed training")
    logging.info('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
    
# Multi-worker configuration
logging.info('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))
NUM_WORKERS = strategy.num_replicas_in_sync
GLOBAL_BATCH_SIZE = args.batch_size * NUM_WORKERS


def _is_chief(task_type, task_id):
    ''' Check for primary if multiworker training
    '''
    return (task_type == 'chief') or (task_type == 'worker' and task_id == 0) or task_type is None

# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size
BUFFER_SIZE = 10000
train_data = data.get_data(buffer=BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
    # Creation of dataset, and model building/compiling need to be within
    # `strategy.scope()`.
    model = model_.get_model(
        tfhub_url="https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/4",
        input_shape=(32, 32, 3),
        n_classes=10,
        lr=args.lr
    )

model = train.train_model(model, train_data, epochs=args.epochs, steps_per_epoch=args.steps)

if args.distribute == 'multiworker':
    task_type, task_id = (strategy.cluster_resolver.task_type,
                              strategy.cluster_resolver.task_id)
else:
    task_type, task_id = None, None
    
# single, mirrored or primary for multiworker
if _is_chief(task_type, task_id):
     model.save(args.model_dir)
# non-primary workers for multi-workers
else:
    # each worker saves their model instance to a unique temp location
    worker_dir = args.model_dir + '/workertemp_' + str(task_id)
    tf.io.gfile.makedirs(worker_dir)
    model.save(worker_dir)

将培训脚本存储在您的云存储桶中

接下来，将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

### 创建和运行自定义训练作业


要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练作业，2）运行这个作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，需要以下参数：

- `display_name`：自定义训练作业的人类可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball形式。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意：* 没有`requirements`参数。您可以在Python包的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "cifar10_" + TIMESTAMP

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_cifar10.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 准备命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`：要传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接：您将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量 `DIRECT = True`），或者
      - 间接：服务将Cloud Storage位置作为环境变量 `AIP_MODEL_DIR` 传递给训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您需要在作业规范中告知服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时期数。
  - `"--steps=" + STEPS`：每个时期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)

EPOCHS = 10
STEPS = 200

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
]

执行自定义训练作业

接下来，您可以通过调用`run`方法运行自定义作业来启动训练作业，具体参数如下：

- `model_display_name`：`Model`资源的可读名称。
- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`：用于计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：附加到工作器复制品的加速器数量。
- `base_output_dir`：写入模型工件的Cloud Storage位置。
- `sync`：是否阻止直到作业完成。

In [ ]:
if TRAIN_GPU:
    model = job.run(
        model_display_name="cifar10_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=False,
    )
else:
    model = job.run(
        model_display_name="cifar10_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

### 列出一项定制的培训工作

In [ ]:
_job = job.list(filter=f"display_name={DISPLAY_NAME}")
print(_job)

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`以阻塞直到自定义训练作业完成。

In [ ]:
model.wait()

### 删除模型

您可以使用方法`delete()`删除模型。

In [ ]:
model.delete()

#### 删除自定义工作

您可以使用delete()方法删除自定义工作。

In [ ]:
job.delete()

! rm -rf custom custom.tar.gz

清理工作

要清理此项目中使用的所有Google Cloud资源，可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil rm -r $BUCKET_URI